In [1]:
import numpy as np 
import pandas as pd 
from catboost import CatBoostClassifier 
from sklearn.model_selection import train_test_split, StratifiedKFold
from tqdm import tqdm
import tensorflow as tf 
import random
from scipy.stats import kurtosis, skew
import time  

In [2]:
train = pd.read_csv('train_features.csv') 
train_labels = pd.read_csv('train_labels.csv') 
test = pd.read_csv('test_features.csv') 
submission = pd.read_csv('sample_submission.csv')

In [3]:
X = tf.reshape(np.array(train.iloc[:,2:]),[-1, 600, 6])
X = np.asarray(X)

In [4]:
y = train_labels['label'].values 

In [5]:
# we have 3125 labels for the original train set 
labels = np.arange(0,3125)

In [6]:
X.shape, y.shape, labels.shape

((3125, 600, 6), (3125,), (3125,))

In [10]:
k = 10 
models = [] 
kfold = StratifiedKFold(n_splits = k, shuffle = True, random_state = 20210808) 
for n_fold, (train_idx, val_idx) in enumerate(kfold.split(X, y)): 
    X_train, X_val = X[train_idx], X[val_idx] 
    y_train, y_val = y[train_idx], y[val_idx] 
    l_train, l_val = labels[train_idx], labels[val_idx]  
        
    ##### augment data #####
    print("... Augmenting Data ...")
    X_augmented = [] 
    y_augmented = []  
    l_augmented = [] 
    idx = 3125
    for i in tqdm(range(X_train.shape[0]), position = 0, leave = True): 
        for j in range(10): 
            shifted = np.roll(X_train[i], int(random.random() * 600)) 
            X_augmented.append(shifted) 
            y_augmented.append(y_train[i])     
            l_augmented.append(idx) 
            idx += 1 
    X_augmented = np.asarray(X_augmented) 
    y_augmented = np.asarray(y_augmented)
    l_augmented = np.asarray(l_augmented) 
    X_train = np.concatenate([X_train, X_augmented]) 
    y_train = np.concatenate([y_train, y_augmented])
    l_train = np.concatenate([l_train, l_augmented])  
    
    
    '''
    ##### feature engineering data ##### 
    print("... Feature Engineering ...")
    X_fourier_real = [] 
    X_fourier_imag = [] 
    for i in tqdm(range(X_train.shape[0]), position = 0, leave = True):  
        real_part = np.fft.fft(X_train[i]).real 
        imag_part = np.fft.fft(X_train[i]).imag 
        X_fourier_real.append(real_part)
        X_fourier_imag.append(imag_part) 
    
    X_fourier_real = np.asarray(X_fourier_real)  
    X_fourier_imag = np.asarray(X_fourier_imag)
    X_train = np.concatenate([X_train, X_fourier_real, X_fourier_imag], axis = 2)   
    
    
    X_val_fourier_real = [] 
    X_val_fourier_imag = [] 
    for i in tqdm(range(X_val.shape[0]), position = 0, leave = True):
        real_part = np.fft.fft(X_val[i]).real 
        imag_part = np.fft.fft(X_val[i]).imag 
        X_val_fourier_real.append(real_part) 
        X_val_fourier_imag.append(imag_part)
    
    X_val_fourier_real = np.asarray(X_val_fourier_real) 
    X_val_fourier_imag = np.asarray(X_val_fourier_imag)
    X_val = np.concatenate([X_val, X_val_fourier_real, X_val_fourier_imag], axis = 2)  
    ''' 
    
    
    ##### create dataframe for X_train and X_val ##### 
    print("... Creating Dataframe ...")
    l_train_flat = [] 
    for i in tqdm(range(len(l_train))):
        for j in range(600): 
            l_train_flat.append(l_train[i]) 
    l_train_flat = np.asarray(l_train_flat)
    
    l_val_flat = [] 
    for i in tqdm(range(len(l_val))): 
        for j in range(600): 
            l_val_flat.append(l_val[i]) 
    l_val_flat = np.asarray(l_val_flat)
    
    start = time.time() 
    print("... Aggregating Train DataFrame ...")
    train_df = pd.DataFrame(np.concatenate(X_train)) 
    train_df.insert(0,'id',l_train_flat) 
    # features = ['id', 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]
    features = ['id',0,1,2,3,4,5]
    train_df_processed = train_df[features].groupby('id').agg(['max', 'min', 'mean', 'std', 'skew']) 
    
    print("... Aggregating Val DataFrame ...") 
    val_df = pd.DataFrame(np.concatenate(X_val)) 
    val_df.insert(0, 'id', l_val_flat)  
    val_df_processed = val_df[features].groupby('id').agg(['max', 'min', 'mean', 'std', 'skew'])   
    
    print("total pre-processing time = {}".format(time.time() - start))  
    
    
    print("... Begin Training ...") 
    print("Validating on fold {}".format(n_fold))

    model = CatBoostClassifier(iterations = 20000, 
                               learning_rate = 0.01, 
                               l2_leaf_reg = 3.5,
                               depth = 8, 
                               loss_function = 'MultiClass',
                               eval_metric = 'MultiClass',
                               use_best_model = True, 
                               task_type = 'GPU',
                               verbose = 200,
                               random_seed = 88888,
                               classes_count = 61) 
    
    
    model.fit(train_df_processed, y_train, eval_set = (val_df_processed, y_val))
    
    model.save_model('augmented_catboost_' + str(n_fold))  # save model checkpoint just in case 
    models.append(model) 


  9%|▉         | 257/2812 [00:00<00:00, 2564.32it/s]

... Augmenting Data ...


  2%|▏         | 753/30932 [00:00<00:04, 7522.24it/s]

... Creating Dataframe ...


100%|██████████| 313/313 [00:00<00:00, 7623.35it/s]


...

... Creating Dataframe ...


100%|██████████| 312/312 [00:00<00:00, 8335.12it/s]


... Aggregating Train DataFrame ...
... Aggregating Val DataFrame ...
total pre-processing time = 38.260475397109985
... Begin Training ...
Validating on fold 9
0:	learn: 3.9288268	test: 3.9228895	best: 3.9228895 (0)	total: 49.6ms	remaining: 16m 32s


200:	learn: 1.8743456	test: 1.8305206	best: 1.8305206 (200)	total: 8.46s	remaining: 13m 53s
400:	learn: 1.4508003	test: 1.5137402	best: 1.5137402 (400)	total: 17.7s	remaining: 14m 24s
600:	learn: 1.1677007	test: 1.3345407	best: 1.3345407 (600)	total: 26.9s	remaining: 14m 29s
800:	learn: 0.9626508	test: 1.2153780	best: 1.2153780 (800)	total: 36.1s	remaining: 14m 24s
1000:	learn: 0.8131238	test: 1.1357413	best: 1.1357413 (1000)	total: 45.2s	remaining: 14m 17s
1200:	learn: 0.6953066	test: 1.0767178	best: 1.0767178 (1200)	total: 54.3s	remaining: 14m 10s
1400:	learn: 0.6023214	test: 1.0343982	best: 1.0343982 (1400)	total: 1m 3s	remaining: 14m 2s
1600:	learn: 0.5239461	test: 0.9984298	best: 0.9984298 (1600)	total: 1m 12s	remaining: 13m 53s
1800:	learn: 0.4610706	test: 0.9704959	best: 0.9704959 (1800)	total: 1m 21s	remaining: 13m 43s
2000:	learn: 0.4100306	test: 0.9492225	best: 0.9492225 (2000)	total: 1m 30s	remaining: 13m 34s
2200:	learn: 0.3687656	test: 0.9315281	best: 0.9315281 (2200)	tota

17600:	learn: 0.0185505	test: 0.8458478	best: 0.8260187 (8601)	total: 12m 57s	remaining: 1m 46s
17800:	learn: 0.0182430	test: 0.8462235	best: 0.8260187 (8601)	total: 13m 6s	remaining: 1m 37s
18000:	learn: 0.0179512	test: 0.8467082	best: 0.8260187 (8601)	total: 13m 15s	remaining: 1m 28s
18200:	learn: 0.0176692	test: 0.8473010	best: 0.8260187 (8601)	total: 13m 24s	remaining: 1m 19s
18400:	learn: 0.0173864	test: 0.8477124	best: 0.8260187 (8601)	total: 13m 33s	remaining: 1m 10s
18600:	learn: 0.0171090	test: 0.8484299	best: 0.8260187 (8601)	total: 13m 41s	remaining: 1m 1s
18800:	learn: 0.0168402	test: 0.8488717	best: 0.8260187 (8601)	total: 13m 50s	remaining: 53s
19000:	learn: 0.0165766	test: 0.8492725	best: 0.8260187 (8601)	total: 13m 59s	remaining: 44.1s
19200:	learn: 0.0163269	test: 0.8497719	best: 0.8260187 (8601)	total: 14m 8s	remaining: 35.3s
19400:	learn: 0.0160750	test: 0.8503688	best: 0.8260187 (8601)	total: 14m 17s	remaining: 26.5s
19600:	learn: 0.0158276	test: 0.8511877	best: 0.8

## Make prediction

In [13]:
features = ['id', 0, 1, 2, 3, 4, 5] 
test = test.rename(columns={'acc_x':0, 'acc_y':1, 'acc_z':2, 'gy_x':3, 'gy_y':4, 'gy_z':5})
X_test = test[features].groupby('id').agg(['max', 'min', 'mean', 'std', 'skew'])
X_test.head() 

0                                                 1            \
           max       min      mean       std      skew       max       min   
id                                                                           
3125 -0.275446 -1.564000 -1.018731  0.236232  0.480681  0.228040 -0.470937   
3126  0.627571 -1.929033 -0.522843  0.539688  0.783894  1.708743 -0.200678   
3127  2.972063 -0.792916  0.506947  0.219934  2.601368  1.941820  0.219008   
3128  0.337281 -1.045889 -0.577603  0.431713  0.632767 -0.258476 -1.294482   
3129  0.015642 -2.153047 -0.738640  0.305797 -0.334159  1.562602 -0.860883   

                                    ...           4                        \
          mean       std      skew  ...         max         min      mean   
id                                  ...                                     
3125 -0.019574  0.091641 -0.738230  ...   96.185341  -81.607713  2.731872   
3126  0.612161  0.333015  0.141712  ...  241.240196  -97.100707  6.974772   
3127  0.903819  0.191485  0.048348  ...   74.530763  -98.420987 -2.251452   
3128 -0.610557  0.233601 -0.709434  ...  118.268797 -168.031080 -2.984969   
3129  0.182535  0.314294  0.244188  ...  226.728939 -223.475411  1.442366   

                                    5                                   \
            std      skew         max         min      mean        std   
id                                                                       
3125  31.993022  0.285366   49.981455  -35.446915 -2.000683  12.251648   
3126  45.706311  1.240907  169.417650 -147.597574 -3.604579  61.604867   
3127  13.467885 -0.950816   97.211730 -154.477074 -0.393175  23.041463   
3128  45.069932 -0.736913  167.860762 -117.297766 -0.024318  37.967372   
3129  67.911174 -0.005484  138.130133 -125.598600  5.745498  43.353007   

                
          skew  
id              
3125  0.284970  
3126  0.462374  
3127 -1.103520  
3128  0.832220  
3129  0.042361  

[5 rows x 30 columns]

In [14]:
pred1 = models[0].predict_proba(X_test) 
pred2 = models[1].predict_proba(X_test) 
pred3 = models[2].predict_proba(X_test) 
pred4 = models[3].predict_proba(X_test) 
pred5 = models[4].predict_proba(X_test)
pred6 = models[5].predict_proba(X_test) 
pred7 = models[6].predict_proba(X_test) 
pred8 = models[7].predict_proba(X_test) 
pred9 = models[8].predict_proba(X_test) 
pred10 = models[9].predict_proba(X_test)

In [15]:
pred_avg = (pred1 + pred2 + pred3 + pred4 + pred5 + pred6 + pred7 + pred8 + pred9 + pred10)/10.0 

In [16]:
pred_avg.shape

(782, 61)

In [17]:
submission.iloc[:,1:] = pred_avg
submission.to_csv('augmented_catboost_10_fold.csv',index=False)  

In [18]:
submission.head()

,id,0,1,2,3,4,5,6,7,8,...,51,52,53,54,55,56,57,58,59,60
0,3125,0.000041,0.000079,0.000049,0.000122,0.000176,0.000006,0.000175,0.000459,0.000017,...,0.005536,0.000980,0.000995,0.000443,0.000083,0.000053,0.000294,0.131575,0.000002,0.000380
1,3126,0.000524,0.000051,0.000009,0.000177,0.000061,0.000068,0.000003,0.000212,0.000051,...,0.000157,0.000005,0.000055,0.000037,0.000040,0.000152,0.000241,0.000018,0.000014,0.000209
2,3127,0.002592,0.186397,0.000201,0.002554,0.000451,0.000292,0.087915,0.019466,0.001650,...,0.000088,0.000112,0.000202,0.007033,0.000340,0.000941,0.000257,0.000221,0.000194,0.013071
3,3128,0.001859,0.000240,0.000316,0.000520,0.000107,0.000112,0.000009,0.000351,0.000106,...,0.000114,0.000014,0.000078,0.000045,0.000261,0.000019,0.001375,0.000033,0.000017,0.015760
4,3129,0.000520,0.000332,0.000060,0.000870,0.001271,0.000206,0.000007,0.000338,0.000304,...,0.000127,0.000003,0.000045,0.000037,0.000278,0.000230,0.000195,0.000012,0.000090,0.000041


## Average the results from GRU and catboost

In [22]:
## average dataframes     
gru_df = pd.read_csv('fourier_transform_10_fold_gru.csv') 
gru_df.head()

,id,0,1,2,3,4,5,6,7,8,...,51,52,53,54,55,56,57,58,59,60
0,3125,0.000035,0.008070,1.592733e-05,0.000018,0.000013,4.770708e-08,0.352726,0.002920,0.000025,...,0.011902,5.910614e-04,0.000436,0.001321,1.436803e-06,0.000013,0.000198,0.122010,0.000005,0.000023
1,3126,0.000126,0.000812,5.921524e-06,0.021470,0.000004,6.673528e-05,0.000018,0.002538,0.000017,...,0.000009,6.659289e-07,0.000004,0.000250,3.787994e-05,0.000515,0.000336,0.000006,0.000032,0.000115
2,3127,0.000486,0.003486,6.210780e-06,0.000033,0.001839,1.618933e-04,0.070613,0.000983,0.000256,...,0.000582,3.588521e-03,0.000041,0.001242,8.133279e-06,0.000025,0.000115,0.019070,0.000449,0.015425
3,3128,0.000218,0.000011,4.616908e-05,0.000032,0.000003,3.350694e-06,0.000014,0.000017,0.000010,...,0.000118,3.847815e-05,0.000034,0.000061,5.054480e-05,0.000014,0.000958,0.000188,0.000006,0.003009
4,3129,0.049436,0.004557,1.857505e-07,0.000031,0.031574,1.093202e-05,0.000067,0.000006,0.000038,...,0.000013,1.700347e-06,0.000002,0.000001,5.227133e-07,0.000014,0.000324,0.000097,0.000037,0.000634


In [24]:
avg_df = pd.read_csv('sample_submission.csv') 
avg_df.iloc[:,1:] = (gru_df.iloc[:,1:] + submission.iloc[:,1:]) / 2.0  

In [28]:
avg_df.head()

,id,0,1,2,3,4,5,6,7,8,...,51,52,53,54,55,56,57,58,59,60
0,3125,0.000038,0.004075,0.000032,0.000070,0.000094,0.000003,0.176451,0.001690,0.000021,...,0.008719,0.000786,0.000715,0.000882,0.000042,0.000033,0.000246,0.126792,0.000003,0.000201
1,3126,0.000325,0.000431,0.000007,0.010823,0.000032,0.000067,0.000011,0.001375,0.000034,...,0.000083,0.000003,0.000030,0.000144,0.000039,0.000333,0.000288,0.000012,0.000023,0.000162
2,3127,0.001539,0.094941,0.000103,0.001294,0.001145,0.000227,0.079264,0.010224,0.000953,...,0.000335,0.001850,0.000122,0.004138,0.000174,0.000483,0.000186,0.009646,0.000321,0.014248
3,3128,0.001038,0.000126,0.000181,0.000276,0.000055,0.000057,0.000012,0.000184,0.000058,...,0.000116,0.000026,0.000056,0.000053,0.000156,0.000016,0.001167,0.000110,0.000011,0.009385
4,3129,0.024978,0.002445,0.000030,0.000451,0.016423,0.000109,0.000037,0.000172,0.000171,...,0.000070,0.000002,0.000024,0.000019,0.000139,0.000122,0.000260,0.000055,0.000063,0.000337


In [29]:
avg_df.to_csv('gru_catboost_10_fold_augmented.csv',index=False)